### Part 1 + 2 - Bert

In [ ]:
from datasets import load_dataset, load_from_disk
import os

dataset_path = 'imdb_subset'
if os.path.exists(dataset_path):
    subset = load_from_disk(dataset_path)
else:
    dataset = load_dataset('imdb')
    subset = dataset['train'].shuffle(seed=42).select(range(500))
    subset.save_to_disk('imdb_subset')

dataset = subset

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification

model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)

def tokenize_function(example):
    return tokenizer(example["text"], padding="max_length", truncation=True)

split_dataset = dataset.train_test_split(test_size=0.2, seed=42)
train_dataset = split_dataset["train"]
test_dataset = split_dataset["test"]

# Tokenize the train and test datasets
tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_test = test_dataset.map(tokenize_function, batched=True)

# Set the format for PyTorch
tokenized_train.set_format("torch", columns=["input_ids", "attention_mask", "label"])
tokenized_test.set_format("torch", columns=["input_ids", "attention_mask", "label"])

# Confirm dataset sizes
print(f"Train size: {len(tokenized_train)}")
print(f"Test size: {len(tokenized_test)}")

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",               # Directory to save model checkpoints
    evaluation_strategy="epoch",          # Evaluate at the end of each epoch
    learning_rate=2e-5,                   # Learning rate
    per_device_train_batch_size=8,        # Batch size for training
    per_device_eval_batch_size=8,         # Batch size for evaluation
    num_train_epochs=5,                   # Number of epochs
    weight_decay=0.01,                    # Weight decay
    logging_dir="./logs",                 # Directory to save logs
    logging_steps=10,                     # Log every 10 steps
    save_total_limit=2                    # Limit the number of saved checkpoints
)

In [ ]:
from transformers import Trainer
import numpy as np
from sklearn.metrics import accuracy_score

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    preds = np.argmax(predictions, axis=1)  # Get the index of the max logit
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc}


trainer = Trainer(
    model=model,                          # The pre-trained model to fine-tune
    args=training_args,                   # Training arguments
    train_dataset=tokenized_train,        # Training dataset
    eval_dataset=tokenized_test,          # Evaluation dataset
    tokenizer=tokenizer,                  # Tokenizer for data preprocessing
    compute_metrics=compute_metrics       # Custom metrics function
)

In [ ]:
trainer.train()

In [ ]:
results = trainer.evaluate()
print(results)

### Part 3 - GPT 2

In [7]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, Trainer, TrainingArguments

model = GPT2LMHeadModel.from_pretrained("gpt2")
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

def tokenize_reviews(dataset, tokenizer, max_length=150):
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
    def tokenize_function(examples):
        return  tokenizer(examples['text'], padding="max_length",
                truncation=True, max_length=max_length)

    tokenized_dataset = dataset.map(tokenize_function, batched=True)
    tokenized_dataset = tokenized_dataset.rename_column("label", "labels")
    return tokenized_dataset

In [ ]:
training_args = TrainingArguments(
    output_dir="./gpt2-finetuned-results",  # Where to store the model
    evaluation_strategy="epoch",           # Evaluate at each epoch
    per_device_train_batch_size=4,         # Adjust to fit your memory
    per_device_eval_batch_size=4,
    learning_rate=5e-5,                    # Typical learning rate for fine-tuning
    num_train_epochs=3,                    # Adjust as needed
    weight_decay=0.01,                     # Helps reduce overfitting
    logging_dir="./logs",                  # Where to store logs
    logging_steps=10,                      # Adjust for more or less logging
    save_total_limit=2,                    # Keep only 2 checkpoints to save space
    save_strategy="epoch",                 # Save checkpoints each epoch
)

In [11]:
from transformers import DataCollatorForLanguageModeling

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    data_collator=DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False),
)

In [ ]:
trainer.train()